### Reference (Optuna 관련)
https://www.dacon.io/codeshare/2750  
https://www.dacon.io/codeshare/2704  
https://ssoonidev.tistory.com/107  

In [11]:
!pip3 install optuna

  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=35554b6730a581c325e688b467690fcfb683dc5bd25c2564a96786eee46deea0
  Stored in directory: c:\users\judyj\appdata\local\pip\cache\wheels\0c\09\9e\49e21a6840ef7955b06d47394afef0058f0378c0914e48b8b8
Successfully built pyperclip


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, KFold

import warnings
warnings.filterwarnings('ignore')

from catboost import CatBoostClassifier, cv, Pool
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, auc, roc_auc_score
from dmba import classificationSummary

import optuna
from optuna import Trial
from optuna.samplers import TPESampler

from imblearn.over_sampling import SMOTE

In [2]:
train_df = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Train.csv')
test_df = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Test.csv')

train_df['ownerChange'] = round(train_df['ownerChange'])
train_df['bedCount_class'] = round(train_df['bedCount_class'])

test_df['ownerChange'] = round(train_df['ownerChange'])
test_df['bedCount_class'] = round(train_df['bedCount_class'])

train_df.drop('instkind_nan', axis=1, inplace=True)
test_df.drop('instkind_nan', axis=1, inplace=True)

train_df.set_index('inst_id', inplace=True)
test_df.set_index('inst_id', inplace=True)

In [3]:
train_lr = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Train_lr.csv')

train_lr['ownerChange'] = round(train_lr['ownerChange'])
train_lr['bedCount_class'] = round(train_lr['bedCount_class'])

train_lr.set_index('inst_id', inplace=True)

train_lr.head()

,sido_choongchung,sido_gyeongsang,sido_jeonra,instkind_nursing_hospital,instkind_traditional_hospital,revenue1,sga1,salary1,noe1,interest1,...,profit2,inventoryAsset2,OnonCAsset2,shortLoan2,NCLiabilities2,longLoan2,surplus2,ownerChange,bedCount_class,OC
inst_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,0.0,22.162515,22.099796,21.433189,16.539187,16.398469,...,19.152229,16.425505,18.798422,0.000000,20.099863,19.781657,20.963246,0.0,3.0,1
3,0.0,1.0,0.0,0.0,0.0,23.980280,23.481393,19.521179,16.823799,15.443340,...,17.025473,16.044684,15.372413,9.180719,18.812726,17.568559,12.258481,0.0,4.0,1
4,0.0,0.0,0.0,1.0,0.0,20.727778,19.918561,19.507228,10.308986,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,4.0,1
7,0.0,0.0,0.0,0.0,0.0,25.006954,24.981392,24.182293,20.954034,20.902985,...,20.546991,20.934483,22.088313,22.944579,23.755157,23.437961,22.769775,0.0,4.0,1
9,0.0,1.0,0.0,0.0,0.0,24.615974,24.587275,23.920337,21.073281,20.991191,...,17.301128,19.908536,20.486709,23.590810,23.784786,23.370010,22.923325,0.0,3.0,1


In [4]:
train_dt = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Train_dt.csv')
train_dt.drop('instkind_nan', axis=1, inplace=True)

train_dt.head()

,inst_id,instkind_hospital,instkind_traditional_clinic,revenue1,salescost1,sga1,salary1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset2,receivableL2,employee2,OC
0,1,0.0,0.0,22.162515,0.000000,22.099796,21.433189,20.735886,20.720935,19.968362,16.425505,0.0,4.174387,1
1,3,0.0,0.0,23.980280,9.659782,23.481393,19.521179,19.074269,19.037270,10.917985,16.044684,0.0,6.701960,1
2,4,0.0,0.0,20.727778,20.060616,19.918561,19.507228,19.422937,19.351593,15.906875,0.000000,0.0,0.693147,1
3,7,0.0,0.0,25.006954,0.000000,24.981392,24.182293,23.291406,23.168630,0.000000,20.934483,0.0,6.498282,1
4,9,0.0,0.0,24.615974,0.000000,24.587275,23.920337,22.566524,22.493677,22.134087,19.908536,0.0,5.288267,1


In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 1. 67개 columns

In [6]:
X_train = train_df.drop('OC', axis=1)
y_train = train_df['OC']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

smote = SMOTE(random_state=42)
X_tr_over, y_tr_over = smote.fit_resample(X_train, y_train)

### 1) accuracy 기준으로 튜닝

In [7]:
def objective1(trial):
    param = {
        "random_state":42,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "n_estimators":trial.suggest_int("n_estimators", 1000, 5000, 10000),
        "max_depth":trial.suggest_int("max_depth", 4, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
        }
        
    cat = CatBoostClassifier(**param)
    cat.fit(X_tr_over, y_tr_over,
            eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
            early_stopping_rounds=35,
            verbose=100)
    pred = cat.predict(X_val)
    accuracy = accuracy_score(y_val, pred)
    
    return accuracy

In [8]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective1, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-09-02 15:38:24,276] A new study created in memory with name: cat_parameter_opt


0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 150ms	remaining: 2m 29s
100:	learn: 0.2334189	test: 0.2334434	test1: 0.3520380	best: 0.3458430 (92)	total: 257ms	remaining: 2.29s


[I 2022-09-02 15:38:24,949] Trial 0 finished with value: 0.9 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.9.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.29736682
bestIteration = 119

Shrink model to first 120 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 643us	remaining: 643ms
100:	learn: 0.4938532	test: 0.4938783	test1: 0.4917815	best: 0.4911899 (98)	total: 109ms	remaining: 973ms
200:	learn: 0.3447439	test: 0.3447570	test1: 0.3543914	best: 0.3543914 (200)	total: 248ms	remaining: 987ms
300:	learn: 0.2628804	test: 0.2629073	test1: 0.2968388	best: 0.2961610 (296)	total: 359ms	remaining: 834ms
400:	learn: 0.1645062	test: 0.1645390	test1: 0.2311673	best: 0.2301117 (387)	total: 518ms	remaining: 776ms
500:	learn: 0.1352238	test: 0.1352555	test1: 0.2102038	best: 0.2100082 (498)	total: 607ms	remaining: 606ms
600:	learn: 0.0958061	test: 0.0958398	test1: 0.1902497	best: 0.1893616 (591)	total: 785ms	remaining: 522ms
700:	learn: 0.0622392	test: 0.0622730	test1: 0.1781886	best: 0.1775142 (687)	total: 913ms	remaining: 391ms
800:	learn: 0

[I 2022-09-02 15:38:26,821] Trial 1 finished with value: 0.95 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.95.


...

Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1940652785
bestIteration = 22

Shrink model to first 23 iterations.
0:	learn: 0.6420873	test: 0.6421417	test1: 0.6502708	best: 0.6502708 (0)	total: 913us	remaining: 912ms


[I 2022-09-02 15:38:39,724] Trial 8 finished with value: 0.9666666666666667 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 3 with value: 0.9666666666666667.


100:	learn: 0.0002463	test: 0.0002473	test1: 0.3018460	best: 0.2070575 (68)	total: 148ms	remaining: 1.32s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2070575464
bestIteration = 68

Shrink model to first 69 iterations.
0:	learn: 0.6460395	test: 0.6460678	test1: 0.6744767	best: 0.6744767 (0)	total: 132ms	remaining: 2m 11s
100:	learn: 0.0034012	test: 0.0034071	test1: 0.1997320	best: 0.1851242 (85)	total: 10.4s	remaining: 1m 32s


[I 2022-09-02 15:38:52,675] Trial 9 finished with value: 0.95 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 3 with value: 0.9666666666666667.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.18512419
bestIteration = 85

Shrink model to first 86 iterations.
Best Score: 0.9666666666666667
Best trial {'learning_rate': 0.07896186801026692, 'bagging_temperature': 0.04809461967501574, 'n_estimators': 1000, 'max_depth': 4, 'random_strength': 95, 'colsample_bylevel': 0.0923915031962725, 'l2_leaf_reg': 2.425383647001267e-05, 'min_child_samples': 34, 'max_bin': 229, 'od_type': 'IncToDec'}


In [9]:
def clf_eval(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    specificity = confusion_matrix(y_test, pred)[0][0]/(confusion_matrix(y_test, pred)[0][0]+confusion_matrix(y_test, pred)[0][1])
    print('오차행렬')
    classificationSummary(y_test, pred)
    print()
    print('정확도 : {:.4f}\n정밀도 : {:.4f}\n재현율(민감도) : {:.4f}'.format(accuracy, precision, recall))
    print('특이도 : {:.4f}'.format(specificity))
    
    try:
        print('AUC : {:.4f}'.format(roc_auc_score(y_test, pred)))
    except ValueError:
        print('AUC : test data에 하나의 레이블만이 존재하여 AUC를 구할 수 없습니다.')

In [10]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
clf_eval(y_val, cat.predict(X_val))

0:	learn: 0.6665275	test: 0.6665514	test1: 0.6676483	best: 0.6676483 (0)	total: 3.96ms	remaining: 3.95s
100:	learn: 0.0141685	test: 0.0141800	test1: 0.2016528	best: 0.1542726 (65)	total: 191ms	remaining: 1.7s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1542725759
bestIteration = 65

Shrink model to first 66 iterations.

오차행렬
Confusion Matrix (Accuracy 0.9500)

       Prediction
Actual  0  1
     0  0  1
     1  2 57

정확도 : 0.9500
정밀도 : 0.9828
재현율(민감도) : 0.9661
특이도 : 0.0000
AUC : 0.4831


In [11]:
clf_eval(y_test, cat.predict(X_test))

오차행렬
Confusion Matrix (Accuracy 0.8852)

       Prediction
Actual  0  1
     0  0  5
     1  2 54

정확도 : 0.8852
정밀도 : 0.9153
재현율(민감도) : 0.9643
특이도 : 0.0000
AUC : 0.4821


### 2) AUC 기준으로 튜닝

In [12]:
def objective2(trial):
    param = {
        "random_state":42,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "n_estimators":trial.suggest_int("n_estimators", 1000, 5000, 10000),
        "max_depth":trial.suggest_int("max_depth", 4, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
        }
        
    cat = CatBoostClassifier(**param)
    cat.fit(X_tr_over, y_tr_over,
            eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
            early_stopping_rounds=35,
            verbose=100)
    pred = cat.predict(X_val)
    
    return roc_auc_score(y_val, cat.predict(X_val))

In [15]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective2, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-09-02 15:44:13,626] A new study created in memory with name: cat_parameter_opt


0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 700us	remaining: 700ms
100:	learn: 0.2334189	test: 0.2334434	test1: 0.3520380	best: 0.3458430 (92)	total: 102ms	remaining: 907ms


[I 2022-09-02 15:44:13,953] Trial 0 finished with value: 0.4576271186440678 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.4576271186440678.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.29736682
bestIteration = 119

Shrink model to first 120 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 815us	remaining: 815ms
100:	learn: 0.4938532	test: 0.4938783	test1: 0.4917815	best: 0.4911899 (98)	total: 102ms	remaining: 906ms
200:	learn: 0.3447439	test: 0.3447570	test1: 0.3543914	best: 0.3543914 (200)	total: 245ms	remaining: 976ms
300:	learn: 0.2628804	test: 0.2629073	test1: 0.2968388	best: 0.2961610 (296)	total: 340ms	remaining: 789ms
400:	learn: 0.1645062	test: 0.1645390	test1: 0.2311673	best: 0.2301117 (387)	total: 485ms	remaining: 726ms
500:	learn: 0.1352238	test: 0.1352555	test1: 0.2102038	best: 0.2100082 (498)	total: 599ms	remaining: 597ms
600:	learn: 0.0958061	test: 0.0958398	test1: 0.1902497	best: 0.1893616 (591)	total: 792ms	remaining: 527ms
700:	learn: 0.0622392	test: 0.0622730	test1: 0.1781886	best: 0.1775142 (687)	total: 926ms	remaining: 396ms
800:	learn: 0

[I 2022-09-02 15:44:15,873] Trial 1 finished with value: 0.4830508474576271 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.4830508474576271.


...

Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1940652785
bestIteration = 22

Shrink model to first 23 iterations.
0:	learn: 0.6420873	test: 0.6421417	test1: 0.6502708	best: 0.6502708 (0)	total: 1.41ms	remaining: 1.41s


[I 2022-09-02 15:44:29,977] Trial 8 finished with value: 0.4915254237288136 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 5 with value: 0.9576271186440678.


100:	learn: 0.0002463	test: 0.0002473	test1: 0.3018460	best: 0.2070575 (68)	total: 262ms	remaining: 2.33s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2070575464
bestIteration = 68

Shrink model to first 69 iterations.
0:	learn: 0.6460395	test: 0.6460678	test1: 0.6744767	best: 0.6744767 (0)	total: 235ms	remaining: 3m 54s
100:	learn: 0.0034012	test: 0.0034071	test1: 0.1997320	best: 0.1851242 (85)	total: 16.9s	remaining: 2m 30s


[I 2022-09-02 15:44:49,728] Trial 9 finished with value: 0.4830508474576271 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 5 with value: 0.9576271186440678.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.18512419
bestIteration = 85

Shrink model to first 86 iterations.
Best Score: 0.9576271186440678
Best trial {'learning_rate': 0.27051668818999286, 'bagging_temperature': 12.60466458564947, 'n_estimators': 1000, 'max_depth': 16, 'random_strength': 90, 'colsample_bylevel': 0.03961867790406585, 'l2_leaf_reg': 2.7657008308343274e-05, 'min_child_samples': 13, 'max_bin': 258, 'od_type': 'Iter'}


In [16]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
clf_eval(y_val, cat.predict(X_val))

0:	learn: 0.5010413	test: 0.5010647	test1: 0.5009170	best: 0.5009170 (0)	total: 16.6ms	remaining: 16.5s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.336652214
bestIteration = 10

Shrink model to first 11 iterations.

오차행렬
Confusion Matrix (Accuracy 0.8833)

       Prediction
Actual  0  1
     0  0  1
     1  6 53

정확도 : 0.8833
정밀도 : 0.9815
재현율(민감도) : 0.8983
특이도 : 0.0000
AUC : 0.4492


In [17]:
clf_eval(y_test, cat.predict(X_test))

오차행렬
Confusion Matrix (Accuracy 0.8689)

       Prediction
Actual  0  1
     0  1  4
     1  4 52

정확도 : 0.8689
정밀도 : 0.9286
재현율(민감도) : 0.9286
특이도 : 0.2000
AUC : 0.5643


# 2) Train_lr

In [18]:
X_train = train_lr.drop('OC', axis=1)
y_train = train_lr['OC']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

smote = SMOTE(random_state=42)
X_tr_over, y_tr_over = smote.fit_resample(X_train, y_train)

### 1) accuracy 기준 튜닝

In [19]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective1, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-09-02 15:45:47,092] A new study created in memory with name: cat_parameter_opt
[I 2022-09-02 15:45:47,252] Trial 0 finished with value: 0.8166666666666667 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.8166666666666667.


0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 736us	remaining: 736ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.525522526
bestIteration = 53

Shrink model to first 54 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 684us	remaining: 684ms
100:	learn: 0.6142991	test: 0.6143388	test1: 0.6025109	best: 0.6025109 (100)	total: 67.3ms	remaining: 599ms
200:	learn: 0.5462015	test: 0.5462451	test1: 0.5432633	best: 0.5429936 (199)	total: 137ms	remaining: 544ms
300:	learn: 0.4760113	test: 0.4760335	test1: 0.4789046	best: 0.4781129 (293)	total: 212ms	remaining: 493ms


[I 2022-09-02 15:45:47,770] Trial 1 finished with value: 0.8666666666666667 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.8666666666666667.


400:	learn: 0.4321197	test: 0.4321276	test1: 0.4547934	best: 0.4544647 (371)	total: 286ms	remaining: 428ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.4376710102
bestIteration = 426

Shrink model to first 427 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 702us	remaining: 702ms
100:	learn: 0.2874036	test: 0.2874282	test1: 0.3668128	best: 0.3514807 (88)	total: 91.1ms	remaining: 811ms
200:	learn: 0.0318793	test: 0.0319030	test1: 0.1385489	best: 0.1385489 (200)	total: 207ms	remaining: 825ms
Stopped by overfitting detector  (35 iterations wait)



[I 2022-09-02 15:45:48,181] Trial 2 finished with value: 0.95 and parameters: {'learning_rate': 0.08012737503998542, 'bagging_temperature': 0.03613894271216528, 'n_estimators': 1000, 'max_depth': 7, 'random_strength': 37, 'colsample_bylevel': 0.028580510658069373, 'l2_leaf_reg': 2.355742708217648e-05, 'min_child_samples': 24, 'max_bin': 354, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.95.


...

Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1641839239
bestIteration = 62

Shrink model to first 63 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 620us	remaining: 619ms
100:	learn: 0.1142307	test: 0.1142613	test1: 0.1914988	best: 0.1851064 (87)	total: 85.9ms	remaining: 765ms


[I 2022-09-02 15:45:50,037] Trial 8 finished with value: 0.95 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.95.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1327629418
bestIteration = 143

Shrink model to first 144 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 629us	remaining: 629ms
100:	learn: 0.1281872	test: 0.1282226	test1: 0.2254477	best: 0.2228611 (96)	total: 797ms	remaining: 7.16s


[I 2022-09-02 15:45:51,708] Trial 9 finished with value: 0.9333333333333333 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 2 with value: 0.95.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2101789637
bestIteration = 145

Shrink model to first 146 iterations.
Best Score: 0.95
Best trial {'learning_rate': 0.08012737503998542, 'bagging_temperature': 0.03613894271216528, 'n_estimators': 1000, 'max_depth': 7, 'random_strength': 37, 'colsample_bylevel': 0.028580510658069373, 'l2_leaf_reg': 2.355742708217648e-05, 'min_child_samples': 24, 'max_bin': 354, 'od_type': 'IncToDec'}


In [20]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
classificationSummary(y_val, cat.predict(X_val))
print('\nAUC:', roc_auc_score(y_val, cat.predict(X_val)))

0:	learn: 0.6679950	test: 0.6680236	test1: 0.6798571	best: 0.6798571 (0)	total: 13.7ms	remaining: 13.7s
100:	learn: 0.1125908	test: 0.1126339	test1: 0.2689188	best: 0.2618295 (86)	total: 104ms	remaining: 925ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2246995311
bestIteration = 140

Shrink model to first 141 iterations.

Confusion Matrix (Accuracy 0.9167)

       Prediction
Actual  0  1
     0  0  1
     1  4 55

AUC: 0.4661016949152542


In [21]:
clf_eval(y_test, cat.predict(X_test))

오차행렬
Confusion Matrix (Accuracy 0.8852)

       Prediction
Actual  0  1
     0  0  5
     1  2 54

정확도 : 0.8852
정밀도 : 0.9153
재현율(민감도) : 0.9643
특이도 : 0.0000
AUC : 0.4821


### 2) AUC 기준 튜닝

In [22]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective2, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-09-02 15:46:18,651] A new study created in memory with name: cat_parameter_opt
[I 2022-09-02 15:46:18,794] Trial 0 finished with value: 0.4152542372881356 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.4152542372881356.


0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 646us	remaining: 646ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.525522526
bestIteration = 53

Shrink model to first 54 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 587us	remaining: 587ms
100:	learn: 0.6142991	test: 0.6143388	test1: 0.6025109	best: 0.6025109 (100)	total: 59.1ms	remaining: 526ms
200:	learn: 0.5462015	test: 0.5462451	test1: 0.5432633	best: 0.5429936 (199)	total: 130ms	remaining: 517ms


[I 2022-09-02 15:46:19,302] Trial 1 finished with value: 0.4406779661016949 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.4406779661016949.


300:	learn: 0.4760113	test: 0.4760335	test1: 0.4789046	best: 0.4781129 (293)	total: 204ms	remaining: 473ms
400:	learn: 0.4321197	test: 0.4321276	test1: 0.4547934	best: 0.4544647 (371)	total: 278ms	remaining: 415ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.4376710102
bestIteration = 426

Shrink model to first 427 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 1.05ms	remaining: 1.05s
100:	learn: 0.2874036	test: 0.2874282	test1: 0.3668128	best: 0.3514807 (88)	total: 89.8ms	remaining: 800ms


[I 2022-09-02 15:46:19,653] Trial 2 finished with value: 0.4830508474576271 and parameters: {'learning_rate': 0.08012737503998542, 'bagging_temperature': 0.03613894271216528, 'n_estimators': 1000, 'max_depth': 7, 'random_strength': 37, 'colsample_bylevel': 0.028580510658069373, 'l2_leaf_reg': 2.355742708217648e-05, 'min_child_samples': 24, 'max_bin': 354, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.4830508474576271.


...

Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1641839239
bestIteration = 62

Shrink model to first 63 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 463us	remaining: 463ms
100:	learn: 0.1142307	test: 0.1142613	test1: 0.1914988	best: 0.1851064 (87)	total: 66.3ms	remaining: 590ms


[I 2022-09-02 15:46:21,437] Trial 8 finished with value: 0.9745762711864407 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 8 with value: 0.9745762711864407.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1327629418
bestIteration = 143

Shrink model to first 144 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 554us	remaining: 554ms
100:	learn: 0.1281872	test: 0.1282226	test1: 0.2254477	best: 0.2228611 (96)	total: 838ms	remaining: 7.53s


[I 2022-09-02 15:46:23,349] Trial 9 finished with value: 0.4745762711864407 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 8 with value: 0.9745762711864407.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2101789637
bestIteration = 145

Shrink model to first 146 iterations.
Best Score: 0.9745762711864407
Best trial {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}


In [24]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
classificationSummary(y_val, cat.predict(X_val))
print('\nAUC:', roc_auc_score(y_val, cat.predict(X_val)))

0:	learn: 0.6380245	test: 0.6380801	test1: 0.6646944	best: 0.6646944 (0)	total: 1.08ms	remaining: 1.08s
100:	learn: 0.0303890	test: 0.0304067	test1: 0.1433092	best: 0.1266855 (72)	total: 75.3ms	remaining: 670ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1266854613
bestIteration = 72

Shrink model to first 73 iterations.

Confusion Matrix (Accuracy 0.9667)

       Prediction
Actual  0  1
     0  0  1
     1  1 58

AUC: 0.4915254237288136


In [25]:
clf_eval(y_test, cat.predict(X_test))

오차행렬
Confusion Matrix (Accuracy 0.9180)

       Prediction
Actual  0  1
     0  0  5
     1  0 56

정확도 : 0.9180
정밀도 : 0.9180
재현율(민감도) : 1.0000
특이도 : 0.0000
AUC : 0.5000


# 3) Train_dt

In [26]:
X_train = train_dt.drop('OC', axis=1)
y_train = train_dt['OC']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

smote = SMOTE(random_state=42)
X_tr_over, y_tr_over = smote.fit_resample(X_train, y_train)

### 1) accuracy 기준

In [27]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective1, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-09-02 15:48:38,206] A new study created in memory with name: cat_parameter_opt


0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 1.24ms	remaining: 1.24s
100:	learn: 0.6349239	test: 0.6349611	test1: 0.6353589	best: 0.6353589 (100)	total: 55.3ms	remaining: 493ms
200:	learn: 0.5871756	test: 0.5872172	test1: 0.5829253	best: 0.5829253 (200)	total: 104ms	remaining: 414ms


[I 2022-09-02 15:48:38,631] Trial 0 finished with value: 0.8666666666666667 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.8666666666666667.


300:	learn: 0.5005499	test: 0.5005610	test1: 0.4919876	best: 0.4919876 (300)	total: 164ms	remaining: 381ms
400:	learn: 0.4256693	test: 0.4256854	test1: 0.4317782	best: 0.4317782 (400)	total: 220ms	remaining: 328ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.4153037179
bestIteration = 420

Shrink model to first 421 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 661us	remaining: 660ms
100:	learn: 0.6698613	test: 0.6698769	test1: 0.6748575	best: 0.6748575 (100)	total: 53.5ms	remaining: 476ms
200:	learn: 0.6503891	test: 0.6504269	test1: 0.6484667	best: 0.6483592 (194)	total: 103ms	remaining: 411ms
300:	learn: 0.6282777	test: 0.6283285	test1: 0.6257651	best: 0.6257651 (300)	total: 155ms	remaining: 360ms
400:	learn: 0.6132464	test: 0.6132970	test1: 0.6125262	best: 0.6125262 (400)	total: 205ms	remaining: 306ms
500:	learn: 0.5972223	test: 0.5972629	test1: 0.5918505	best: 0.5918505 (500)	total: 263ms	remaining: 262ms
600:	lear

[I 2022-09-02 15:48:39,545] Trial 1 finished with value: 0.85 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.8666666666666667.


...

200:	learn: 0.0605579	test: 0.0605785	test1: 0.2598560	best: 0.2343627 (166)	total: 176ms	remaining: 700ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2343627074
bestIteration = 166

Shrink model to first 167 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 604us	remaining: 604ms
100:	learn: 0.3185655	test: 0.3185941	test1: 0.3678340	best: 0.3678340 (100)	total: 62.1ms	remaining: 553ms
200:	learn: 0.2470327	test: 0.2470670	test1: 0.3234685	best: 0.3230624 (191)	total: 117ms	remaining: 463ms
300:	learn: 0.1977074	test: 0.1977371	test1: 0.2870464	best: 0.2869113 (294)	total: 173ms	remaining: 402ms
400:	learn: 0.1559487	test: 0.1559771	test1: 0.2372124	best: 0.2371889 (398)	total: 230ms	remaining: 343ms


[I 2022-09-02 15:48:42,276] Trial 8 finished with value: 0.95 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 3 with value: 0.95.


500:	learn: 0.0889884	test: 0.0890083	test1: 0.1952885	best: 0.1944527 (483)	total: 291ms	remaining: 290ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1944526508
bestIteration = 483

Shrink model to first 484 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 571us	remaining: 571ms
100:	learn: 0.5349846	test: 0.5350208	test1: 0.5412904	best: 0.5407327 (87)	total: 89.8ms	remaining: 799ms
200:	learn: 0.4317623	test: 0.4317854	test1: 0.4551514	best: 0.4551514 (200)	total: 186ms	remaining: 739ms
300:	learn: 0.3296377	test: 0.3296662	test1: 0.3687577	best: 0.3677265 (286)	total: 327ms	remaining: 759ms
400:	learn: 0.2701405	test: 0.2701694	test1: 0.3195351	best: 0.3193901 (398)	total: 413ms	remaining: 618ms
500:	learn: 0.2329516	test: 0.2329790	test1: 0.2864079	best: 0.2864079 (500)	total: 499ms	remaining: 497ms
600:	learn: 0.2126520	test: 0.2126804	test1: 0.2708944	best: 0.2708476 (582)	total: 576ms	remaining: 382ms
700:	learn

[I 2022-09-02 15:48:43,412] Trial 9 finished with value: 0.9333333333333333 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 3 with value: 0.95.


800:	learn: 0.1458179	test: 0.1458486	test1: 0.2309680	best: 0.2309615 (799)	total: 743ms	remaining: 185ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2205253176
bestIteration = 822

Shrink model to first 823 iterations.
Best Score: 0.95
Best trial {'learning_rate': 0.07896186801026692, 'bagging_temperature': 0.04809461967501574, 'n_estimators': 1000, 'max_depth': 4, 'random_strength': 95, 'colsample_bylevel': 0.0923915031962725, 'l2_leaf_reg': 2.425383647001267e-05, 'min_child_samples': 34, 'max_bin': 229, 'od_type': 'IncToDec'}


In [28]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
classificationSummary(y_val, cat.predict(X_val))
print('\nAUC:', roc_auc_score(y_val, cat.predict(X_val)))

0:	learn: 0.6811663	test: 0.6811727	test1: 0.6871124	best: 0.6871124 (0)	total: 1.05ms	remaining: 1.05s
100:	learn: 0.2194073	test: 0.2194296	test1: 0.3388927	best: 0.3379586 (68)	total: 103ms	remaining: 915ms
200:	learn: 0.0424431	test: 0.0424620	test1: 0.2762875	best: 0.2640982 (177)	total: 202ms	remaining: 802ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2640982313
bestIteration = 177

Shrink model to first 178 iterations.

Confusion Matrix (Accuracy 0.9667)

       Prediction
Actual  0  1
     0  0  1
     1  1 58

AUC: 0.4915254237288136


In [29]:
clf_eval(y_test, cat.predict(X_test))

오차행렬
Confusion Matrix (Accuracy 0.9016)

       Prediction
Actual  0  1
     0  0  5
     1  1 55

정확도 : 0.9016
정밀도 : 0.9167
재현율(민감도) : 0.9821
특이도 : 0.0000
AUC : 0.4911


### 2) AUC 기준

In [30]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective2, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-09-02 15:49:05,741] A new study created in memory with name: cat_parameter_opt


0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 653us	remaining: 653ms
100:	learn: 0.6349239	test: 0.6349611	test1: 0.6353589	best: 0.6353589 (100)	total: 52.8ms	remaining: 470ms
200:	learn: 0.5871756	test: 0.5872172	test1: 0.5829253	best: 0.5829253 (200)	total: 107ms	remaining: 427ms


[I 2022-09-02 15:49:06,195] Trial 0 finished with value: 0.4406779661016949 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.4406779661016949.


300:	learn: 0.5005499	test: 0.5005610	test1: 0.4919876	best: 0.4919876 (300)	total: 169ms	remaining: 392ms
400:	learn: 0.4256693	test: 0.4256854	test1: 0.4317782	best: 0.4317782 (400)	total: 234ms	remaining: 350ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.4153037179
bestIteration = 420

Shrink model to first 421 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 591us	remaining: 591ms
100:	learn: 0.6698613	test: 0.6698769	test1: 0.6748575	best: 0.6748575 (100)	total: 61.3ms	remaining: 545ms
200:	learn: 0.6503891	test: 0.6504269	test1: 0.6484667	best: 0.6483592 (194)	total: 119ms	remaining: 475ms
300:	learn: 0.6282777	test: 0.6283285	test1: 0.6257651	best: 0.6257651 (300)	total: 185ms	remaining: 430ms
400:	learn: 0.6132464	test: 0.6132970	test1: 0.6125262	best: 0.6125262 (400)	total: 261ms	remaining: 389ms
500:	learn: 0.5972223	test: 0.5972629	test1: 0.5918505	best: 0.5918505 (500)	total: 335ms	remaining: 333ms
600:	lear

[I 2022-09-02 15:49:07,231] Trial 1 finished with value: 0.4322033898305085 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.4406779661016949.


...

200:	learn: 0.0605579	test: 0.0605785	test1: 0.2598560	best: 0.2343627 (166)	total: 144ms	remaining: 571ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2343627074
bestIteration = 166

Shrink model to first 167 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 579us	remaining: 579ms
100:	learn: 0.3185655	test: 0.3185941	test1: 0.3678340	best: 0.3678340 (100)	total: 54.9ms	remaining: 489ms
200:	learn: 0.2470327	test: 0.2470670	test1: 0.3234685	best: 0.3230624 (191)	total: 107ms	remaining: 426ms


[I 2022-09-02 15:49:09,899] Trial 8 finished with value: 0.4830508474576271 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 6 with value: 0.9067796610169492.


300:	learn: 0.1977074	test: 0.1977371	test1: 0.2870464	best: 0.2869113 (294)	total: 162ms	remaining: 376ms
400:	learn: 0.1559487	test: 0.1559771	test1: 0.2372124	best: 0.2371889 (398)	total: 217ms	remaining: 325ms
500:	learn: 0.0889884	test: 0.0890083	test1: 0.1952885	best: 0.1944527 (483)	total: 273ms	remaining: 272ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.1944526508
bestIteration = 483

Shrink model to first 484 iterations.
0:	learn: 0.6931466	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 748us	remaining: 747ms
100:	learn: 0.5349846	test: 0.5350208	test1: 0.5412904	best: 0.5407327 (87)	total: 106ms	remaining: 945ms
200:	learn: 0.4317623	test: 0.4317854	test1: 0.4551514	best: 0.4551514 (200)	total: 195ms	remaining: 776ms
300:	learn: 0.3296377	test: 0.3296662	test1: 0.3687577	best: 0.3677265 (286)	total: 359ms	remaining: 834ms
400:	learn: 0.2701405	test: 0.2701694	test1: 0.3195351	best: 0.3193901 (398)	total: 455ms	remaining: 680ms
500:	learn:

[I 2022-09-02 15:49:11,034] Trial 9 finished with value: 0.4745762711864407 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 6 with value: 0.9067796610169492.


700:	learn: 0.1800598	test: 0.1800887	test1: 0.2473833	best: 0.2469772 (695)	total: 687ms	remaining: 293ms
800:	learn: 0.1458179	test: 0.1458486	test1: 0.2309680	best: 0.2309615 (799)	total: 764ms	remaining: 190ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2205253176
bestIteration = 822

Shrink model to first 823 iterations.
Best Score: 0.9067796610169492
Best trial {'learning_rate': 0.03750796359625606, 'bagging_temperature': 0.12172958098369972, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 36, 'colsample_bylevel': 0.01909565280104538, 'l2_leaf_reg': 1.6285455533915874e-05, 'min_child_samples': 18, 'max_bin': 441, 'od_type': 'Iter'}


In [31]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
classificationSummary(y_val, cat.predict(X_val))
print('\nAUC:', roc_auc_score(y_val, cat.predict(X_val)))

0:	learn: 0.6884495	test: 0.6884594	test1: 0.6875993	best: 0.6875993 (0)	total: 1.44ms	remaining: 1.43s
100:	learn: 0.6033441	test: 0.6033825	test1: 0.6106074	best: 0.6106074 (100)	total: 55.2ms	remaining: 492ms
200:	learn: 0.5418323	test: 0.5418446	test1: 0.5387500	best: 0.5387500 (200)	total: 116ms	remaining: 462ms
300:	learn: 0.4628652	test: 0.4628874	test1: 0.4524357	best: 0.4524357 (300)	total: 184ms	remaining: 428ms
400:	learn: 0.4069684	test: 0.4069944	test1: 0.4148368	best: 0.4147605 (392)	total: 245ms	remaining: 366ms
500:	learn: 0.3528777	test: 0.3529068	test1: 0.3774650	best: 0.3774650 (500)	total: 307ms	remaining: 306ms
600:	learn: 0.3247245	test: 0.3247541	test1: 0.3653151	best: 0.3652395 (594)	total: 375ms	remaining: 249ms
700:	learn: 0.2899703	test: 0.2899971	test1: 0.3340233	best: 0.3340233 (700)	total: 438ms	remaining: 187ms
800:	learn: 0.2481404	test: 0.2481670	test1: 0.3085429	best: 0.3047557 (786)	total: 505ms	remaining: 126ms
Stopped by overfitting detector  (35 it

In [32]:
clf_eval(y_test, cat.predict(X_test))

오차행렬
Confusion Matrix (Accuracy 0.8689)

       Prediction
Actual  0  1
     0  0  5
     1  3 53

정확도 : 0.8689
정밀도 : 0.9138
재현율(민감도) : 0.9464
특이도 : 0.0000
AUC : 0.4732
